In [1]:
import os 
import json
import psycopg2
from datetime import datetime

In [3]:
#establishing connection to db
conn = psycopg2.connect("host=10.0.0.3 port=5432 dbname=pid-portal user=postgres password=passwd") 
cur = conn.cursor()

In [14]:
cur.execute("""select avg_delay from "Trip_delay" where trip_id='332_638_190831' and day_nr=1""") #just simple select to check if connection is working
cur.fetchone() is None

True

# Vehicle positions


In [4]:
def digest(file):
    f = open(file, 'r')
    raw_data = f.read()
    #raw_data.replace('{"features":', '/n{"features":')
    #raw_data.replace('/n{"features":', '', 1)
    #lines = raw_data.split('\n')
    d = '}{'
    lines =  ['{' + e + '}' for e in raw_data.split(d) if e]
    
    lines[0] = lines[0].replace('{{', '{', 1)
    lines[-1] = lines[-1].replace('"type":"Feature"}],"type":"FeatureCollection"}}', '"type":"Feature"}],"type":"FeatureCollection"}')
    
    data = []
    for line in lines:
        if line != '':
            try:
                data.append(json.loads(line))
            except:
                exc_file = open('err_line', 'a')
                exc_file.write(line + '\n')
                exc_file.close()
        
    
    #df = pd.read_json(file, lines=True)
    return data


In [5]:

def convert_unix_to_date(unix_ts):
    try:
        return datetime.fromtimestamp(int(unix_ts[:-3])).strftime('%Y-%m-%d %H:%M:%S')
    except:
        return unix_ts
    

In [6]:
def insert_single_file(entry):
    i = 0
    for line in entry:
        for data in line['features']:
            cur.execute("""
                INSERT INTO public."Vehicle_position"(
                route_id, "timestamp", bearing, calculated_delay, speed, longitude, latitude, vehicle_id, trip_id, is_canceled, next_stop_id, delay_stop_departure, delay_stop_arrival, distance_traveled)
                VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s);
                """,
               (data['properties']['trip']['gtfs_route_id'],convert_unix_to_date(data['properties']['last_position']['origin_timestamp']), 
                data['properties']['last_position']['bearing'], data['properties']['last_position']['delay'],
                data['properties']['last_position']['speed'], data['properties']['last_position']['lng'], data['properties']['last_position']['lat'],
                data['properties']['trip']['cis_vehicle_registration_number'], data['properties']['trip']['gtfs_trip_id'], data['properties']['last_position']['is_canceled'],
                data['properties']['last_position']['gtfs_next_stop_id'], 
                data['properties']['last_position']['delay_stop_departure'],
                data['properties']['last_position']['delay_stop_arrival'],
                data['properties']['last_position']['gtfs_shape_dist_traveled']
               ))
        #trip delays
            if data['properties']['last_position']['delay_stop_departure'] != None:
                cur.execute("""
        INSERT INTO public."Trip_delay"(
		trip_id, day_nr, avg_delay, cnt)
		VALUES (%s, extract(isodow from date %s), %s, 1) ON CONFLICT (trip_id,day_nr) DO UPDATE 
	  SET avg_delay = (%s + "Trip_delay".cnt * "Trip_delay".avg_delay)/("Trip_delay".cnt+1), cnt = "Trip_delay".cnt+1 ;
        """, (data['properties']['trip']['gtfs_trip_id'], convert_unix_to_date(data['properties']['last_position']['origin_timestamp'][:-3]), data['properties']['last_position']['delay_stop_departure'],
         data['properties']['last_position']['delay_stop_departure']))
        #section delays
                cur.execute("""
        UPDATE public."Trip_sections"
		SET last_delay=%s,
		avg_delay = case when avg_delay is null then %s else (%s+("Trip_sections".delay_counter*"Trip_sections".avg_delay))/("Trip_sections".delay_counter+1) end,
		delay_counter = case when delay_counter is null then 1 else delay_counter+1 end
		WHERE trip_id=%s and destination=%s;
        """, (data['properties']['last_position']['delay_stop_departure'], data['properties']['last_position']['delay_stop_departure'], data['properties']['last_position']['delay_stop_departure'], data['properties']['trip']['gtfs_trip_id'], data['properties']['last_position']['gtfs_next_stop_id']))
        #filling the vehicle table from the position data
            cur.execute("""
        INSERT INTO public."Vehicle"(
	vehicle_id, vehicle_type, agency)
	VALUES (%s, %s, %s)
        ON CONFLICT (vehicle_id) DO UPDATE 
        SET vehicle_type = excluded.vehicle_type, agency = excluded.agency;
        """, (data['properties']['trip']['cis_vehicle_registration_number'], data['properties']['trip']['vehicle_type'], data['properties']['trip']['cis_agency_name'])
                   )
        i += 1
    print('Executed {} entries for table Vehicle_position'.format(i))

In [9]:
#entries = [digest(entry) for entry in os.scandir('/mnt/d/data_diplomka/vehicle_positions/') if entry.is_file()]
entries = []
i = 0
for entry in os.scandir('/mnt/c/data_diplomka/vehicle_position_test/'):
    if entry.is_file():
        #data = digest(entry)
        #break
        print('Now working on file: {}'.format(entry))
        insert_single_file(digest(entry))
        conn.commit()
        

Now working on file: <DirEntry '20051154001520571'>
Executed 118 entries for table Vehicle_position


In [9]:
#conn.rollback()

# Stops

In [ ]:
def get_bool_from_int_for_wheelchair(bool_val):
    if bool_val == 1:
        return "1"
    else:
        return "0"

In [ ]:
def digest_stops(file):
    f = open(file, 'r')
    data = [json.loads(line) for line in open(file, 'r')]
    #data = json.load(f)
    return data

In [ ]:
entries = [digest_stops(entry) for entry in os.scandir('/mnt/c/data_diplomka/stops') if entry.is_file()]

In [ ]:
entries[1][0]['properties']

In [ ]:
i = 0
for file in entries:
    for a in file:
    
        cur.execute("""
    INSERT INTO public."Stop"(
	stop_id, name, latitude, longitude, zone_id, wheelchair_acc)
	VALUES (%s, %s, %s, %s, %s, %s);
    """,(a['properties']['stop_id'], a['properties']['stop_name'], a['properties']['stop_lat'], a['properties']['stop_lon'], a['properties']['zone_id'], 
        get_bool_from_int_for_wheelchair(a['properties']['wheelchair_boarding'])
        ))
        i += 1
    print('Executed {} entries for table Stop'.format(i))

In [ ]:
conn.commit()

# Routes

In [ ]:
def digest_routes(file):
    f = open(file, 'r')
    data = [json.loads(line) for line in open(file, 'r')]
    #data = json.load(f)
    return data

In [ ]:
entries = [digest_routes(entry) for entry in os.scandir('/mnt/c/data_diplomka/shapes') if entry.is_file()]

In [ ]:
entries[1][0]['features'][0]

In [ ]:
i = 0
for route in entries:
    for point in route[0]['features']:
        cur.execute("""
        INSERT INTO public."Route"(
        route_id, "order", distance, latitude, longitude, stop_id)
        VALUES (%s, %s, %s, %s, %s, %s);
        """, 
                    (point['properties']['shape_id'], point['properties']['shape_pt_sequence'], point['properties']['shape_dist_traveled'], point['properties']['shape_pt_lat'],
                    point['properties']['shape_pt_lon'], '')
                   )
        i += 1
        
print('Executed {} entries for stops'.format(i))

In [ ]:
conn.commit()

# Trip

In [ ]:
def digest_trips(file):
    f = open(file, 'r')
    data = [json.loads(line) for line in open(file, 'r')]
    #data = json.load(f)
    return data

In [ ]:
entries = [digest_trips(entry) for entry in os.scandir('/mnt/c/data_diplomka/trips') if entry.is_file()]

In [ ]:
entries[0][1]

In [ ]:
for file in entries:
    for entry in file:
        cur.execute("""
        INSERT INTO public."Trip"(
	trip_id, route_id, shape_id, direction, bikes_allowed, headsign)
	VALUES (%s, %s, %s, %s, %s, %s);""", 
                    (entry['trip_id'], entry['route_id'], entry['shape_id'], get_bool_from_int_for_wheelchair(entry['direction_id']), get_bool_from_int_for_wheelchair(entry['bikes_allowed']),
                    entry['trip_headsign'])
                    )

In [ ]:
conn.commit()

# Stop Times

In [ ]:
def digest_st_times(file):
    f = open(file, 'r')
    data = [json.loads(line) for line in open(file, 'r')]
    #data = json.load(f)
    return data

In [ ]:
entries = [digest_st_times(entry) for entry in os.scandir('/mnt/c/data_diplomka/stop_times') if entry.is_file()]

In [ ]:
entries[1][0]

In [ ]:
for file in entries:
    for entry in file:
        cur.execute("""
        INSERT INTO public.stop_time(
	trip_id, stop_id, sequence, arival_time, departure_time)
	VALUES (%s, %s, %s, %s, %s)
    ON CONFLICT (trip_id, stop_id, sequence) DO UPDATE 
  SET trip_id = excluded.trip_id, 
  stop_id = excluded.stop_id,
  sequence = excluded.sequence,
  arival_time = excluded.arival_time,
  departure_time = excluded.departure_time
  ;""", 
                    (entry['trip_id'], entry['stop_id'], entry['stop_sequence'], entry['arrival_time'],
                    entry['departure_time'])
                    )

In [ ]:
conn.commit() 

# Route_sections

In [ ]:
cur.execute("""
INSERT INTO public."Trip_sections" (source, destination, sequence, trip_id)
(select st1.stop_id as source, st2.stop_id as destination, st1.sequence, st1.trip_id
from "stop_time" as st1, "stop_time" as st2 
where st1.trip_id=st2.trip_id and st2.sequence = st1.sequence+1 
 
) on conflict DO NOTHING;
""")

conn.commit()

In [ ]:
entries = []

In [ ]:
conn.rollback()